# compiler

## compiler 是整个webpack 的核心， 控制着整个webpack的生命周期

In [ ]:
// compiler 钩子函数

this.hooks = Object.freeze({
    /** @type {SyncHook<[]>} */
    initialize: new SyncHook([]),

    /** @type {SyncBailHook<[Compilation], boolean>} */
    shouldEmit: new SyncBailHook(["compilation"]),
    /** @type {AsyncSeriesHook<[Stats]>} */
    done: new AsyncSeriesHook(["stats"]),
    /** @type {SyncHook<[Stats]>} */
    afterDone: new SyncHook(["stats"]),
    /** @type {AsyncSeriesHook<[]>} */
    additionalPass: new AsyncSeriesHook([]),
    /** @type {AsyncSeriesHook<[Compiler]>} */
    beforeRun: new AsyncSeriesHook(["compiler"]),
    /** @type {AsyncSeriesHook<[Compiler]>} */
    run: new AsyncSeriesHook(["compiler"]),
    /** @type {AsyncSeriesHook<[Compilation]>} */
    emit: new AsyncSeriesHook(["compilation"]),
    /** @type {AsyncSeriesHook<[string, AssetEmittedInfo]>} */
    assetEmitted: new AsyncSeriesHook(["file", "info"]),
    /** @type {AsyncSeriesHook<[Compilation]>} */
    afterEmit: new AsyncSeriesHook(["compilation"]),

    /** @type {SyncHook<[Compilation, CompilationParams]>} */
    thisCompilation: new SyncHook(["compilation", "params"]),
    /** @type {SyncHook<[Compilation, CompilationParams]>} */
    compilation: new SyncHook(["compilation", "params"]),
    /** @type {SyncHook<[NormalModuleFactory]>} */
    normalModuleFactory: new SyncHook(["normalModuleFactory"]),
    /** @type {SyncHook<[ContextModuleFactory]>}  */
    contextModuleFactory: new SyncHook(["contextModuleFactory"]),

    /** @type {AsyncSeriesHook<[CompilationParams]>} */
    beforeCompile: new AsyncSeriesHook(["params"]),
    /** @type {SyncHook<[CompilationParams]>} */
    compile: new SyncHook(["params"]),
    /** @type {AsyncParallelHook<[Compilation]>} */
    make: new AsyncParallelHook(["compilation"]),
    /** @type {AsyncParallelHook<[Compilation]>} */
    finishMake: new AsyncSeriesHook(["compilation"]),
    /** @type {AsyncSeriesHook<[Compilation]>} */
    afterCompile: new AsyncSeriesHook(["compilation"]),

    /** @type {AsyncSeriesHook<[]>} */
    readRecords: new AsyncSeriesHook([]),
    /** @type {AsyncSeriesHook<[]>} */
    emitRecords: new AsyncSeriesHook([]),

    /** @type {AsyncSeriesHook<[Compiler]>} */
    watchRun: new AsyncSeriesHook(["compiler"]),
    /** @type {SyncHook<[Error]>} */
    failed: new SyncHook(["error"]),
    /** @type {SyncHook<[string | null, number]>} */
    invalid: new SyncHook(["filename", "changeTime"]),
    /** @type {SyncHook<[]>} */
    watchClose: new SyncHook([]),
    /** @type {AsyncSeriesHook<[]>} */
    shutdown: new AsyncSeriesHook([]),

    /** @type {SyncBailHook<[string, string, any[]], true>} */
    infrastructureLog: new SyncBailHook(["origin", "type", "args"]),

    // TODO the following hooks are weirdly located here
    // TODO move them for webpack 5
    /** @type {SyncHook<[]>} */
    environment: new SyncHook([]),
    /** @type {SyncHook<[]>} */
    afterEnvironment: new SyncHook([]),
    /** @type {SyncHook<[Compiler]>} */
    afterPlugins: new SyncHook(["compiler"]),
    /** @type {SyncHook<[Compiler]>} */
    afterResolvers: new SyncHook(["compiler"]),
    /** @type {SyncBailHook<[string, Entry], boolean>} */
    entryOption: new SyncBailHook(["context", "entry"])
});

## compiler run 函数

In [ ]:
run(callback){
        const run = () => {
            /**
                依次出发 beforeRun, run, 通知各个插件
                然后运行readRecords 和 compile 开始编译
            **/
            this.hooks.beforeRun.callAsync(this, err => {
				if (err) return finalCallback(err);

				this.hooks.run.callAsync(this, err => {
					if (err) return finalCallback(err);

					this.readRecords(err => {
						if (err) return finalCallback(err);

						this.compile(onCompiled);
					});
				});
			});
		};
        // 先判断当前是否正在运行中
        // 如果运行中，出发end事件，然后在重新运行
		if (this.idle) {
			this.cache.endIdle(err => {
				if (err) return finalCallback(err);

				this.idle = false;
				run();
			});
		} else {
			run();
		}
}

## compiler compile 函数

In [ ]:
/**
    依次执行钩子函数 beforeCompile compile make finishMake afterCompile
    
    
**/

compile(callback) {
    const params = this.newCompilationParams();
    this.hooks.beforeCompile.callAsync(params, err => {
        if (err) return callback(err);

        this.hooks.compile.call(params);

        const compilation = this.newCompilation(params);

        const logger = compilation.getLogger("webpack.Compiler");

        logger.time("make hook");
        this.hooks.make.callAsync(compilation, err => {
            logger.timeEnd("make hook");
            if (err) return callback(err);

            logger.time("finish make hook");
            this.hooks.finishMake.callAsync(compilation, err => {
                logger.timeEnd("finish make hook");
                if (err) return callback(err);

                process.nextTick(() => {
                    logger.time("finish compilation");
                    compilation.finish(err => {
                        logger.timeEnd("finish compilation");
                        if (err) return callback(err);

                        logger.time("seal compilation");
                        compilation.seal(err => {
                            logger.timeEnd("seal compilation");
                            if (err) return callback(err);

                            logger.time("afterCompile hook");
                            this.hooks.afterCompile.callAsync(compilation, err => {
                                logger.timeEnd("afterCompile hook");
                                if (err) return callback(err);

                                return callback(null, compilation);
                            });
                        });
                    });
                });
            });
        });
    });
}

主要关注点：
`const params = this.newCompilationParams();`
`const compilation = this.newCompilation(params);`

In [ ]:
newCompilationParams() {
    // 这两个都是解析module的
    const params = {
        // 针对正常的node_modules
        normalModuleFactory: this.createNormalModuleFactory(),
        // 针对文件
        contextModuleFactory: this.createContextModuleFactory()
    };
    return params;
}